In [2]:
import numpy as np

class LinearRegression():
    def __init__(self, regresor, response, has_bias = False, regularizer = None):
        if (regresor.ndim != 2) or (regresor.size == 0):
            raise ValueError("regresor should be a non-empty numpy matrix")
        elif (response.ndim != 2) or (response.size == 0):
            raise ValueError("response should be a non-empty numpy matrix")
        elif len(regresor) != len(response):
            raise ValueError("len(regresor) != len(response)")
        
        self.has_bias = has_bias
        if self.has_bias:
            self.regresor = np.append(regresor, np.ones((len(regresor), 1)), axis=1)
        else:
            self.regresor = regresor
        
        self.response = response
        self.regularizer = regularizer